In [6]:
! pip install datasets

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import torch
import os
import pandas as pd
from collections import Counter
import string

/Users/bochnak/anaconda3/envs/nlp2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Test dataset

questions = (
    pd.read_json("questions.jl", lines=True)
    .rename(columns={"_id": "question-id"})
    .sort_index()
)

questions["question-id"] = questions["question-id"].astype(int)

relevant = pd.read_json("relevant.jl", lines=True).sort_index()
relevant["question-id"] = relevant["question-id"].astype(int)

passages = (
    pd.read_json("passages.jl", lines=True)
    .rename(columns={"_id": "passage-id"})
    .sort_index()
)

answers = pd.read_json("answers.jl", lines=True).set_index("question-id").sort_index()

df = (
    questions.merge(relevant, on="question-id")
    .merge(passages, on="passage-id")
    .merge(answers, on="question-id")
    .rename(columns={"text_x": "question", "text_y": "context"})
)

In [3]:
# PoQuAD has a python API compatible with the datasets library, but it only provides the extractive answers, have to read JSON files directly

# ds = load_dataset("clarin-pl/poquad")
# df_train = pd.DataFrame(ds["train"])
# df_train.head()
import json

with open("poquad-dev.json") as f:
    poquad_dev = json.load(f)["data"]

with open("poquad-train.json") as f:
    poquad_train = json.load(f)["data"]

poquad_dev[0]
poquad_train[0]

{'id': 7609,
 'title': 'Konfederacja polsko-czechosłowacka',
 'summary': 'Konfederacja polsko-czechosłowacka – koncepcja polityczna z okresu II wojny światowej propagowana przez Rząd RP na uchodźstwie. Idea ta, wspierana przez rząd Wielkiej Brytanii, nawiązywała do przedwojennych planów ustanowienia bloku państw „Międzymorza”. Początkowo częściami składowymi konfederacji miały być Polska, Czechosłowacja i Węgry.\nW okresie II wojny światowej rządy niektórych państw europejskich na uchodźstwie dokonywały prób jednoczenia się w związki, które miały dać im w przyszłości gwarancję bezpieczeństwa, trwałości politycznej i gospodarczej. Odżywały koncepcje federacyjne z lat 20. i 30. XX w., które w nowych warunkach miały szansę realizacji. Wiele projektów federalistycznych można odnaleźć w programach polskich stronnictw politycznych w okresie II wojny światowej. Takie próby były dokonywane przez Polskę, Czechosłowację, Grecję i Jugosławię. Projekt federacji (konfederacji) polsko-czechosłowacki

In [4]:
def process_poquad(data):
    contexts, questions, answers = [], [], []
    for article in data:
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                # have problem with inconsistency in data
                if "answers" not in qa or not qa["answers"]:
                    continue
                answer = qa["answers"][0]["text"]
                contexts.append(context)
                questions.append(question)
                answers.append(answer)
    return pd.DataFrame({"context": contexts, "question": questions, "answer": answers})


train_df = process_poquad(poquad_train)
val_df = process_poquad(poquad_dev)

In [6]:
train_df[:2]

,context,question,answer
0,Projekty konfederacji zaczęły się załamywać 5 ...,Co było powodem powrócenia konceptu porozumien...,wymianą listów Ripka – Stroński
1,Projekty konfederacji zaczęły się załamywać 5 ...,Pomiędzy jakimi stronami odbyło się zgromadzen...,E. Beneša i J. Masaryka z jednej a Wł. Sikorsk...


In [7]:
val_df[:2]

,context,question,answer
0,Pisma rabiniczne – w tym Miszna – stanowią kom...,Czym są pisma rabiniczne?,kompilację poglądów różnych rabinów na określo...
1,Pisma rabiniczne – w tym Miszna – stanowią kom...,Z ilu komponentów składała się Tora przekazana...,dwóch


In [8]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("allegro/plt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("allegro/plt5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
def preprocess(row):
    inputs = f"Pytanie: {row['question']} Konteks: {row['context']}"
    targets = row["answer"]
    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        targets, max_length=128, truncation=True, padding="max_length"
    ).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_train = train_df.apply(preprocess, axis=1).tolist()
tokenized_val = val_df.apply(preprocess, axis=1).tolist()

In [10]:
train_dataset = Dataset.from_dict(
    {
        "input_ids": [d["input_ids"] for d in tokenized_train],
        "attention_mask": [d["attention_mask"] for d in tokenized_train],
        "labels": [d["labels"] for d in tokenized_train],
    }
)

val_dataset = Dataset.from_dict(
    {
        "input_ids": [d["input_ids"] for d in tokenized_val],
        "attention_mask": [d["attention_mask"] for d in tokenized_val],
        "labels": [d["labels"] for d in tokenized_val],
    }
)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
num_gpus = torch.cuda.device_count()
num_gpus

1

In [13]:
# had some problems with, wandb api key and GPU OOM
os.environ["WANDB_DISABLED"] = "true"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="model_results",
    evaluation_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-14-d992a9d3d51c>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan


In [ ]:
# I used up all the GPU limits in google colab

In [ ]:
results = trainer.evaluate()
print(results)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("apohllo/plt5-base-poquad")
model = AutoModelForSeq2SeqLM.from_pretrained("apohllo/plt5-base-poquad")

In [6]:
@torch.no_grad()
def generate_answer(question: str, context: str):
    input = f"question: {question} context: {context}"

    encoded_input = tokenizer(
        [input], return_tensors="pt", max_length=500, truncation=True
    )

    model.eval()
    output = model.generate(
        input_ids=encoded_input.input_ids,
        attention_mask=encoded_input.attention_mask,
        max_length=500,
    )

    output = tokenizer.decode(
        output[0],
        skip_special_tokens=True,
    )

    return output

In [7]:
def purge_answer(s):
    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))


def f1_score(ground_truth, prediction):
    prediction_tokens = purge_answer(prediction).split()
    ground_truth_tokens = purge_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return purge_answer(prediction) == purge_answer(ground_truth)

In [8]:
# Test dataset
predictions = [generate_answer(r["question"], r["context"]) for i, r in df.iterrows()]

In [9]:
df_pred = df.copy()
df_pred["pred"] = predictions
df_pred["f1"] = df_pred.apply(lambda r: f1_score(r["answer"], r["pred"]), axis=1)
df_pred["exact_match"] = df_pred.apply(
    lambda r: exact_match_score(r["answer"], r["pred"]), axis=1
)

In [10]:
f1 = df_pred.f1.mean() * 100
exact_match = df_pred.exact_match.mean() * 100

print("Results on test dataset:")
print(f"F1 score         : {f1:.3f}")
print(f"exact_match score: {exact_match:.3f}")

Results on test dataset:
F1 score         : 44.561
exact_match score: 24.523


In [11]:
# Validate dataset
predictions = [
    generate_answer(r["question"], r["context"]) for i, r in val_df.iterrows()
]

In [12]:
val_df_pred = val_df.copy()
val_df_pred["pred"] = predictions
val_df_pred["f1"] = val_df_pred.apply(
    lambda r: f1_score(r["answer"], r["pred"]), axis=1
)
val_df_pred["exact_match"] = val_df_pred.apply(
    lambda r: exact_match_score(r["answer"], r["pred"]), axis=1
)

In [13]:
f1 = val_df_pred.f1.mean() * 100
exact_match = val_df_pred.exact_match.mean() * 100

print("Results on test dataset:")
print(f"F1 score         : {f1:.3f}")
print(f"exact_match score: {exact_match:.3f}")

Results on test dataset:
F1 score         : 59.516
exact_match score: 41.794


In [14]:
# Report of best results obtained on validation and test dataset
data = {
    "Dataset": ["Validation", "Test"],
    "F1 Score (%)": [59.516, 44.561],
    "Exact Match Score (%)": [24.523, 24.523],
}

# Create a DataFrame
df = pd.DataFrame(data)
df

,Dataset,F1 Score (%),Exact Match Score (%)
0,Validation,59.516,24.523
1,Test,44.561,24.523


In [18]:
# Report and analyze 10 questions on test datset
df_pred[50:70]

,question-id,question,passage-id,score_x,title,context,score_y,answer,pred,f1,exact_match
50,51,Na jakiej wartości akcje dzieli się kapitał sp...,1996_465_474,1,Ustawa z dnia 15 września 2000 r. Kodeks spółe...,Art. 474. §1. Podział między akcjonariuszy maj...,1.0,proporcjonalnie do dokonanych przez każdego z ...,na kapitał zakładowy,0.428571,False
51,52,Na ile wydawna jest licencja na statek rybacki?,2001_1441_6,1,Ustawa z dnia 6 września 2001 r. o rybołówstwi...,"Art. 6. 1. Licencję, na wniosek armatora, wyda...",1.0,na okres od 1 roku do 5 lat,od 1 roku do 5 lat,0.857143,False
52,53,Jakie zadania wykonują jednostki podległe Szef...,2000_6_1,1,Ustawa z dnia 22 grudnia 1999 r. o czasowym po...,Art. 1. 1. Nadwiślańskie Jednostki Wojskowe Mi...,1.0,zadania w zakresie ochrony osób oraz obiektów ...,w zakresie ochrony osób oraz obiektów o szczeg...,0.800000,False
53,54,Kiedy Skarb Państwa może rozwiązać spółkę celową?,2004_624_5,1,Ustawa z dnia 29 kwietnia 2016 r. o szczególny...,Art. 5. 1. Skarb Państwa rozwiązuje spółkę cel...,1.0,"po zrealizowaniu celu, dla którego została utw...",w przypadku niewykonywania lub nienależytego w...,0.666667,False
54,55,Czy funkcjonariusze Inspekcji Celnej mogą być ...,1997_449_34,1,Ustawa z dnia 6 czerwca 1997 r. o Inspekcji Ce...,Art. 34. 1. Funkcjonariusze Inspekcji Celnej n...,1.0,nie,nie,1.000000,True
55,56,Czy zamniętych zakładach karnych skazani mogą ...,1997_557_90,1,Ustawa z dnia 6 czerwca 1997 r. Kodeks karny w...,Art. 90. W zakładzie karnym typu zamkniętego: ...,1.0,tak,tak,1.000000,True
56,57,Kto tworzy okręgowe inspektoraty rybołówstwa m...,2004_574_52,1,Ustawa z dnia 19 lutego 2004 r. o rybołówstwie...,Art. 52. 1. Okręgowi inspektorzy rybołówstwa m...,1.0,minister właściwy do spraw rolnictwa,minister właściwy do spraw rolnictwa,1.000000,True
57,58,Na jaki okres może zostać pozbawiona wolności ...,1997_553_288,1,Ustawa z dnia 6 czerwca 1997 r. Kodeks karny C...,"Art. 288. § 1. Kto cudzą rzecz niszczy, uszkad...",1.0,od 3 miesięcy do lat 5,od 3 miesięcy do lat 5,1.000000,True
58,59,Jakiej karze podlega naruszanie czyjejś nietyk...,1997_553_257,1,Ustawa z dnia 6 czerwca 1997 r. Kodeks karny C...,Art. 257. Kto publicznie znieważa grupę ludnoś...,1.0,karze pozbawienia wolności do lat 3,pozbawienia wolności do lat 3,0.909091,False
59,60,Które miasto jest siedzibą Krajowego Związku Kas?,1997_153_107,1,Ustawa z dnia 6 lutego 1997 r. o powszechnym u...,Art. 107. 1. Krajowy Związek Kas zrzesza regio...,1.0,Warszawa,Warszawa,1.000000,True


The model performs well on straightforward questions, achieving exact matches for ~47% and high F1 scores. Strengths include excelling at simpler questions (e.g., IDs 54, 55, 56, 59, 60) and interpreting clear regulatory or legislative contexts. Errors arise with nuanced questions due to partial or incorrect phrasing. Improvements are needed for handling ambiguity and semantic variations.

## Questions

##### **1. Does the performance on the validation dataset reflects the performance on your test set?**

Performance on the validation dataset does not fully reflect the performance on the test set. While the mT5-base model achieved an F1 score of 59.5% and an exact match of 24.5% on the validation set, it performed worse on the test set with an F1 score of 44.5% and the same exact match of 24.5%. This indicates that the model generalizes less effectively to unseen data.

##### **2. What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason for that?**


The model's performance on the test questions is decent, with most answers being correct for the selected questions. However, it tends to favor brief responses, often defaulting to simple answers like "Yes" or "No." This could indicate a bias towards concise outputs, potentially due to training data or model limitations in generating more detailed responses.

#### **3. Why extractive question answering is not well suited for inflectional languages?**

Extractive question answering faces challenges in inflectional languages due to their complex word forms, grammatical variations, and flexible word order. In these languages, words can take various forms depending on their usage, and the order of words can change without altering the meaning of the sentence. These complexities make it difficult for extractive systems to reliably align questions with text, as direct keyword matching becomes less effective. The inherent variations in grammar and word forms in inflectional languages complicate the extraction of relevant information, making accurate question answering more challenging.